In [1]:
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import ast
%matplotlib inline
sns.set_theme()

In [2]:
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [3]:
from src.strategy import predict_position
from src.update_files import update_state, update_portfolio
from src.position_sizing import portfolio_calculator, amount_calculator
from src.risk_management import slippage, stop_loss, take_profit
from src.backtester import data_handler, place_order, execute_order, execute_SL_TP
from src.utils import get_config, read_file

In [4]:
config = get_config.read_yaml()
symbols = config['data']['symbols']
symbols

['ETH/USDT',
 'BTC/USDT',
 'BNB/USDT',
 'SOL/USDT',
 'ADA/USDT',
 'LINK/USDT',
 'XRP/USDT',
 'LTC/USDT',
 'XLM/USDT']

In [5]:
test_data = read_file.read_merged_test_data()
test_data

,"('open', 'ETH')","('high', 'ETH')","('low', 'ETH')","('close', 'ETH')","('volume', 'ETH')","('rsi', 'ETH')","('sma-50', 'ETH')","('sma-100', 'ETH')","('sma-200', 'ETH')","('ema-50', 'ETH')",...,"('volume', 'XLM')","('rsi', 'XLM')","('sma-50', 'XLM')","('sma-100', 'XLM')","('sma-200', 'XLM')","('ema-50', 'XLM')","('ema-100', 'XLM')","('ema-200', 'XLM')","('atr', 'XLM')","('adx', 'XLM')"
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-04-01 00:00:00,0.645975,0.790280,0.715671,0.815987,0.404417,0.799613,0.035243,0.0,0.000000,0.007454,...,0.279384,0.486711,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.388106,0.000000
2025-04-01 01:00:00,0.815921,0.609533,0.760587,0.716559,0.111552,0.702352,0.035647,0.0,0.000000,0.002003,...,0.060792,0.438075,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.338228,0.000000
2025-04-01 02:00:00,0.716637,0.722804,0.808499,0.949034,0.057982,0.898482,0.046699,0.0,0.000000,0.020532,...,0.390564,0.841367,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.336147,0.000000
2025-04-01 03:00:00,0.949195,0.754766,0.932126,0.870708,0.062721,0.818193,0.049332,0.0,0.000000,0.032678,...,0.070252,0.697066,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.148579,0.000000
2025-04-01 04:00:00,0.870841,0.744112,0.903881,0.926681,0.000000,0.866355,0.051882,0.0,0.000000,0.053491,...,0.000000,0.798996,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.077985,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-30 19:00:00,0.137176,0.444005,0.137738,0.480312,0.669037,0.507006,1.000000,1.0,0.000000,0.954804,...,0.181166,0.618959,1.0,1.0,0.109477,0.551242,0.792392,0.328975,0.754155,0.065275
2025-09-30 20:00:00,0.480353,0.645392,0.465195,0.757993,0.612325,0.711322,1.000000,1.0,0.000518,0.990312,...,0.178035,0.828181,1.0,1.0,0.156346,0.551255,0.817525,0.311839,0.669875,0.059036
2025-09-30 21:00:00,0.758012,0.675813,0.497405,0.422350,0.559736,0.440226,1.000000,1.0,0.000000,1.000000,...,0.274440,0.613881,1.0,1.0,0.185185,0.516457,0.815188,0.255737,0.755820,0.017864


In [6]:
update_state.set_state(config['strategy']['capital'])
state = read_file.read_state()
state

{'timestep': 0,
 'cash': 100000,
 'metrics': {'transaction_cost': 0.0,
  'returns': 0.0,
  'peak equity': 0.0,
  'drawdown': 0.0}}

In [7]:
candle = data_handler.fetch_data(test_data)
candle

{'ADA': {'adx': 0.0,
  'atr': 0.4288063864571997,
  'close': 0.8170346745414924,
  'ema-100': 0.0,
  'ema-200': 0.0,
  'ema-50': 0.0,
  'high': 0.7230215567258423,
  'low': 0.8388059451102681,
  'open': 0.8468468214160115,
  'rsi': 0.904111480950504,
  'sma-100': 0.0,
  'sma-200': 0.0,
  'sma-50': 0.0,
  'volume': 0.2154082771635632},
 'BNB': {'adx': 0.0,
  'atr': 0.1518530004574088,
  'close': 0.9110005527412434,
  'ema-100': 0.0,
  'ema-200': 0.0,
  'ema-50': 0.1244130749946054,
  'high': 0.8833876220922899,
  'low': 0.9733924611433872,
  'open': 0.9883977900006452,
  'rsi': 0.8527474550609166,
  'sma-100': 0.0,
  'sma-200': 0.0,
  'sma-50': 0.1096642675646819,
  'volume': 0.0335664313563457},
 'BTC': {'adx': 0.0,
  'atr': 0.6309600784635432,
  'close': 0.6109904090817135,
  'ema-100': 0.0,
  'ema-200': 0.0,
  'ema-50': 0.0347136342511214,
  'high': 0.4179619365089645,
  'low': 0.5594711570331365,
  'open': 0.584999926599473,
  'rsi': 0.6336068483723716,
  'sma-100': 0.0,
  'sma-200'

In [8]:
execute_SL_TP.execute(candle)

In [9]:
candle = predict_position.predict(candle)
candle

{'ADA': {'adx': 0.0,
  'atr': 0.4288063864571997,
  'close': 0.8170346745414924,
  'ema-100': 0.0,
  'ema-200': 0.0,
  'ema-50': 0.0,
  'high': 0.7230215567258423,
  'low': 0.8388059451102681,
  'open': 0.8468468214160115,
  'rsi': 0.904111480950504,
  'sma-100': 0.0,
  'sma-200': 0.0,
  'sma-50': 0.0,
  'volume': 0.2154082771635632,
  'fiducia': 0.08},
 'BNB': {'adx': 0.0,
  'atr': 0.1518530004574088,
  'close': 0.9110005527412434,
  'ema-100': 0.0,
  'ema-200': 0.0,
  'ema-50': 0.1244130749946054,
  'high': 0.8833876220922899,
  'low': 0.9733924611433872,
  'open': 0.9883977900006452,
  'rsi': 0.8527474550609166,
  'sma-100': 0.0,
  'sma-200': 0.0,
  'sma-50': 0.1096642675646819,
  'volume': 0.0335664313563457,
  'fiducia': 0.08},
 'BTC': {'adx': 0.0,
  'atr': 0.6309600784635432,
  'close': 0.6109904090817135,
  'ema-100': 0.0,
  'ema-200': 0.0,
  'ema-50': 0.0347136342511214,
  'high': 0.4179619365089645,
  'low': 0.5594711570331365,
  'open': 0.584999926599473,
  'rsi': 0.633606848

In [10]:
update_portfolio.set_portfolio()
portfolio = read_file.read_portfolio()
portfolio

,amt,order_price,stop_price,stop_portion,take_price,take_portion
symbol,,,,,,
ETH,0.0,0.0,0.0,0.0,0.0,0.0
BTC,0.0,0.0,0.0,0.0,0.0,0.0
BNB,0.0,0.0,0.0,0.0,0.0,0.0
SOL,0.0,0.0,0.0,0.0,0.0,0.0
ADA,0.0,0.0,0.0,0.0,0.0,0.0
LINK,0.0,0.0,0.0,0.0,0.0,0.0
XRP,0.0,0.0,0.0,0.0,0.0,0.0
LTC,0.0,0.0,0.0,0.0,0.0,0.0
XLM,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
Pt = portfolio_calculator.calculate(candle)
Pt

np.float64(216578.1907855288)

In [12]:
candle = slippage.get_order_price(candle, Pt)
candle

{'ADA': {'adx': 0.0,
  'atr': 0.4288063864571997,
  'close': 0.8170346745414924,
  'ema-100': 0.0,
  'ema-200': 0.0,
  'ema-50': 0.0,
  'high': 0.7230215567258423,
  'low': 0.8388059451102681,
  'open': 0.8468468214160115,
  'rsi': 0.904111480950504,
  'sma-100': 0.0,
  'sma-200': 0.0,
  'sma-50': 0.0,
  'volume': 0.2154082771635632,
  'fiducia': 0.08,
  'order_price': 0.8186687438905753},
 'BNB': {'adx': 0.0,
  'atr': 0.1518530004574088,
  'close': 0.9110005527412434,
  'ema-100': 0.0,
  'ema-200': 0.0,
  'ema-50': 0.1244130749946054,
  'high': 0.8833876220922899,
  'low': 0.9733924611433872,
  'open': 0.9883977900006452,
  'rsi': 0.8527474550609166,
  'sma-100': 0.0,
  'sma-200': 0.0,
  'sma-50': 0.1096642675646819,
  'volume': 0.0335664313563457,
  'fiducia': 0.08,
  'order_price': 0.9128225538467258},
 'BTC': {'adx': 0.0,
  'atr': 0.6309600784635432,
  'close': 0.6109904090817135,
  'ema-100': 0.0,
  'ema-200': 0.0,
  'ema-50': 0.0347136342511214,
  'high': 0.4179619365089645,
  'l

In [13]:
candle = amount_calculator.calculate(candle, Pt)
candle

{'ADA': {'adx': 0.0,
  'atr': 0.4288063864571997,
  'close': 0.8170346745414924,
  'ema-100': 0.0,
  'ema-200': 0.0,
  'ema-50': 0.0,
  'high': 0.7230215567258423,
  'low': 0.8388059451102681,
  'open': 0.8468468214160115,
  'rsi': 0.904111480950504,
  'sma-100': 0.0,
  'sma-200': 0.0,
  'sma-50': 0.0,
  'volume': 0.2154082771635632,
  'fiducia': 0.08,
  'order_price': 0.8186687438905753,
  'amt': np.float64(21142.796089011703)},
 'BNB': {'adx': 0.0,
  'atr': 0.1518530004574088,
  'close': 0.9110005527412434,
  'ema-100': 0.0,
  'ema-200': 0.0,
  'ema-50': 0.1244130749946054,
  'high': 0.8833876220922899,
  'low': 0.9733924611433872,
  'open': 0.9883977900006452,
  'rsi': 0.8527474550609166,
  'sma-100': 0.0,
  'sma-200': 0.0,
  'sma-50': 0.1096642675646819,
  'volume': 0.0335664313563457,
  'fiducia': 0.08,
  'order_price': 0.9128225538467258,
  'amt': np.float64(18962.005532821404)},
 'BTC': {'adx': 0.0,
  'atr': 0.6309600784635432,
  'close': 0.6109904090817135,
  'ema-100': 0.0,
  

In [14]:
candle = stop_loss.get_stop_loss(candle)
candle

{'ADA': {'adx': 0.0,
  'atr': 0.4288063864571997,
  'close': 0.8170346745414924,
  'ema-100': 0.0,
  'ema-200': 0.0,
  'ema-50': 0.0,
  'high': 0.7230215567258423,
  'low': 0.8388059451102681,
  'open': 0.8468468214160115,
  'rsi': 0.904111480950504,
  'sma-100': 0.0,
  'sma-200': 0.0,
  'sma-50': 0.0,
  'volume': 0.2154082771635632,
  'fiducia': 0.08,
  'order_price': 0.8186687438905753,
  'amt': np.float64(21142.796089011703),
  'stop_price': -1.1109599951668234,
  'stop_portion': np.float64(14799.957262308191)},
 'BNB': {'adx': 0.0,
  'atr': 0.1518530004574088,
  'close': 0.9110005527412434,
  'ema-100': 0.0,
  'ema-200': 0.0,
  'ema-50': 0.1244130749946054,
  'high': 0.8833876220922899,
  'low': 0.9733924611433872,
  'open': 0.9883977900006452,
  'rsi': 0.8527474550609166,
  'sma-100': 0.0,
  'sma-200': 0.0,
  'sma-50': 0.1096642675646819,
  'volume': 0.0335664313563457,
  'fiducia': 0.08,
  'order_price': 0.9128225538467258,
  'amt': np.float64(18962.005532821404),
  'stop_price':

In [15]:
candle = take_profit.get_take_profit(candle)
candle

{'ADA': {'adx': 0.0,
  'atr': 0.4288063864571997,
  'close': 0.8170346745414924,
  'ema-100': 0.0,
  'ema-200': 0.0,
  'ema-50': 0.0,
  'high': 0.7230215567258423,
  'low': 0.8388059451102681,
  'open': 0.8468468214160115,
  'rsi': 0.904111480950504,
  'sma-100': 0.0,
  'sma-200': 0.0,
  'sma-50': 0.0,
  'volume': 0.2154082771635632,
  'fiducia': 0.08,
  'order_price': 0.8186687438905753,
  'amt': np.float64(21142.796089011703),
  'stop_price': -1.1109599951668234,
  'stop_portion': np.float64(14799.957262308191),
  'take_price': 3.3915070626337736,
  'take_portion': np.float64(14799.957262308191)},
 'BNB': {'adx': 0.0,
  'atr': 0.1518530004574088,
  'close': 0.9110005527412434,
  'ema-100': 0.0,
  'ema-200': 0.0,
  'ema-50': 0.1244130749946054,
  'high': 0.8833876220922899,
  'low': 0.9733924611433872,
  'open': 0.9883977900006452,
  'rsi': 0.8527474550609166,
  'sma-100': 0.0,
  'sma-200': 0.0,
  'sma-50': 0.1096642675646819,
  'volume': 0.0335664313563457,
  'fiducia': 0.08,
  'orde

In [16]:
order = place_order.place(candle)
order

{'ADA': {'order_amt': np.float64(21142.796089011703),
  'order_price': 0.8186687438905753,
  'stop_price': -1.1109599951668234,
  'stop_portion': np.float64(14799.957262308191),
  'take_price': 3.3915070626337736,
  'take_portion': np.float64(14799.957262308191)},
 'BNB': {'order_amt': np.float64(18962.005532821404),
  'order_price': 0.9128225538467258,
  'stop_price': 0.22948405178838627,
  'stop_portion': np.float64(13273.403872974983),
  'take_price': 1.8239405565911786,
  'take_portion': np.float64(13273.403872974983)},
 'BTC': {'order_amt': np.float64(77750.14546541679),
  'order_price': 0.6122123898998769,
  'stop_price': -2.2271079631860675,
  'stop_portion': np.float64(54425.10182579175),
  'take_price': 4.397972860681136,
  'take_portion': np.float64(54425.10182579175)},
 'ETH': {'order_amt': np.float64(66295.4043149769),
  'order_price': 0.7179924892575487,
  'stop_price': -0.847405403400223,
  'stop_portion': np.float64(46406.783020483825),
  'take_price': 2.8051896794679108

In [17]:
execute_order.execute(order)
portfolio = read_file.read_portfolio()
portfolio

,amt,order_price,stop_price,stop_portion,take_price,take_portion
symbol,,,,,,
ETH,66295.404315,0.717992,-0.847405,46406.783020,2.805190,46406.783020
BTC,77750.145465,0.612212,-2.227108,54425.101826,4.397973,54425.101826
BNB,18962.005533,0.912823,0.229484,13273.403873,1.823941,13273.403873
SOL,74355.015444,0.232788,-1.626048,52048.510811,2.711236,52048.510811
ADA,21142.796089,0.818669,-1.110960,14799.957262,3.391507,14799.957262
LINK,19472.957237,0.888871,-0.602708,13631.070066,2.877642,13631.070066
XRP,42633.743685,0.405992,-1.325754,29843.620580,2.714986,29843.620580
LTC,52359.664548,0.330578,-1.821772,36651.765184,3.200378,36651.765184
XLM,45218.280946,0.382786,-1.139241,31652.796662,2.412156,31652.796662


In [18]:
state = read_file.read_state()
state

{'timestep': 2,
 'cash': -2.1827872842550278e-11,
 'metrics': {'transaction_cost': 99.5763659150017,
  'returns': 1.0794727484454318,
  'peak equity': 207947.2748445432,
  'drawdown': 0.0}}

In [19]:
candle2 = data_handler.fetch_data(test_data)
candle2

{'ADA': {'adx': 0.0,
  'atr': 0.4231709869658272,
  'close': 0.9999999719887964,
  'ema-100': 0.0,
  'ema-200': 0.0,
  'ema-50': 0.0149480311101181,
  'high': 0.9784172309921844,
  'low': 0.9741935169615022,
  'open': 0.813880100508514,
  'rsi': 0.9999999999675274,
  'sma-100': 0.0,
  'sma-200': 0.0,
  'sma-50': 0.0,
  'volume': 0.1783493411785848},
 'BNB': {'adx': 0.0,
  'atr': 0.071462079737586,
  'close': 0.9999999999475616,
  'ema-100': 0.0,
  'ema-200': 0.0,
  'ema-50': 0.1646964539115449,
  'high': 0.9628664494486774,
  'low': 0.9999999999454744,
  'open': 0.9104972375187564,
  'rsi': 0.9999999999613928,
  'sma-100': 0.0,
  'sma-200': 0.0,
  'sma-50': 0.1402165541294383,
  'volume': 0.1033622385225038},
 'BTC': {'adx': 0.0,
  'atr': 0.5782150029112367,
  'close': 0.8046682325304367,
  'ema-100': 0.0,
  'ema-200': 0.0,
  'ema-50': 0.0602873568034706,
  'high': 0.6012054730800594,
  'low': 0.6491301177202853,
  'open': 0.6109885055218472,
  'rsi': 0.7901603971881096,
  'sma-100': 0

In [20]:
# execute sltp
execute_SL_TP.execute(candle2)

In [21]:
candle2 = predict_position.predict(candle2)
candle2

{'ADA': {'adx': 0.0,
  'atr': 0.4231709869658272,
  'close': 0.9999999719887964,
  'ema-100': 0.0,
  'ema-200': 0.0,
  'ema-50': 0.0149480311101181,
  'high': 0.9784172309921844,
  'low': 0.9741935169615022,
  'open': 0.813880100508514,
  'rsi': 0.9999999999675274,
  'sma-100': 0.0,
  'sma-200': 0.0,
  'sma-50': 0.0,
  'volume': 0.1783493411785848,
  'fiducia': 0.08},
 'BNB': {'adx': 0.0,
  'atr': 0.071462079737586,
  'close': 0.9999999999475616,
  'ema-100': 0.0,
  'ema-200': 0.0,
  'ema-50': 0.1646964539115449,
  'high': 0.9628664494486774,
  'low': 0.9999999999454744,
  'open': 0.9104972375187564,
  'rsi': 0.9999999999613928,
  'sma-100': 0.0,
  'sma-200': 0.0,
  'sma-50': 0.1402165541294383,
  'volume': 0.1033622385225038,
  'fiducia': 0.08},
 'BTC': {'adx': 0.0,
  'atr': 0.5782150029112367,
  'close': 0.8046682325304367,
  'ema-100': 0.0,
  'ema-200': 0.0,
  'ema-50': 0.0602873568034706,
  'high': 0.6012054730800594,
  'low': 0.6491301177202853,
  'open': 0.6109885055218472,
  'rs

In [22]:
Pt = portfolio_calculator.calculate(candle)
Pt

np.float64(215929.96901853514)

In [23]:
candle2 = slippage.get_order_price(candle2, Pt)
candle2

{'ADA': {'adx': 0.0,
  'atr': 0.4231709869658272,
  'close': 0.9999999719887964,
  'ema-100': 0.0,
  'ema-200': 0.0,
  'ema-50': 0.0149480311101181,
  'high': 0.9784172309921844,
  'low': 0.9741935169615022,
  'open': 0.813880100508514,
  'rsi': 0.9999999999675274,
  'sma-100': 0.0,
  'sma-200': 0.0,
  'sma-50': 0.0,
  'volume': 0.1783493411785848,
  'fiducia': 0.08,
  'order_price': 0.9979999720448188},
 'BNB': {'adx': 0.0,
  'atr': 0.071462079737586,
  'close': 0.9999999999475616,
  'ema-100': 0.0,
  'ema-200': 0.0,
  'ema-50': 0.1646964539115449,
  'high': 0.9628664494486774,
  'low': 0.9999999999454744,
  'open': 0.9104972375187564,
  'rsi': 0.9999999999613928,
  'sma-100': 0.0,
  'sma-200': 0.0,
  'sma-50': 0.1402165541294383,
  'volume': 0.1033622385225038,
  'fiducia': 0.08,
  'order_price': 0.9979999999476665},
 'BTC': {'adx': 0.0,
  'atr': 0.5782150029112367,
  'close': 0.8046682325304367,
  'ema-100': 0.0,
  'ema-200': 0.0,
  'ema-50': 0.0602873568034706,
  'high': 0.60120547

In [24]:
candle2 = amount_calculator.calculate(candle2, Pt)
candle2

{'ADA': {'adx': 0.0,
  'atr': 0.4231709869658272,
  'close': 0.9999999719887964,
  'ema-100': 0.0,
  'ema-200': 0.0,
  'ema-50': 0.0149480311101181,
  'high': 0.9784172309921844,
  'low': 0.9741935169615022,
  'open': 0.813880100508514,
  'rsi': 0.9999999999675274,
  'sma-100': 0.0,
  'sma-200': 0.0,
  'sma-50': 0.0,
  'volume': 0.1783493411785848,
  'fiducia': 0.08,
  'order_price': 0.9979999720448188,
  'amt': np.float64(17291.72431312124)},
 'BNB': {'adx': 0.0,
  'atr': 0.071462079737586,
  'close': 0.9999999999475616,
  'ema-100': 0.0,
  'ema-200': 0.0,
  'ema-50': 0.1646964539115449,
  'high': 0.9628664494486774,
  'low': 0.9999999999454744,
  'open': 0.9104972375187564,
  'rsi': 0.9999999999613928,
  'sma-100': 0.0,
  'sma-200': 0.0,
  'sma-50': 0.1402165541294383,
  'volume': 0.1033622385225038,
  'fiducia': 0.08,
  'order_price': 0.9979999999476665,
  'amt': np.float64(17291.723829665978)},
 'BTC': {'adx': 0.0,
  'atr': 0.5782150029112367,
  'close': 0.8046682325304367,
  'ema-

In [25]:
candle2 = stop_loss.get_stop_loss(candle2)
candle2 = take_profit.get_take_profit(candle2)
candle2

{'ADA': {'adx': 0.0,
  'atr': 0.4231709869658272,
  'close': 0.9999999719887964,
  'ema-100': 0.0,
  'ema-200': 0.0,
  'ema-50': 0.0149480311101181,
  'high': 0.9784172309921844,
  'low': 0.9741935169615022,
  'open': 0.813880100508514,
  'rsi': 0.9999999999675274,
  'sma-100': 0.0,
  'sma-200': 0.0,
  'sma-50': 0.0,
  'volume': 0.1783493411785848,
  'fiducia': 0.08,
  'order_price': 0.9979999720448188,
  'amt': np.float64(17291.72431312124),
  'stop_price': -0.9062694693014036,
  'stop_portion': np.float64(12104.207019184869),
  'take_price': 3.5370258938397825,
  'take_portion': np.float64(12104.207019184869)},
 'BNB': {'adx': 0.0,
  'atr': 0.071462079737586,
  'close': 0.9999999999475616,
  'ema-100': 0.0,
  'ema-200': 0.0,
  'ema-50': 0.1646964539115449,
  'high': 0.9628664494486774,
  'low': 0.9999999999454744,
  'open': 0.9104972375187564,
  'rsi': 0.9999999999613928,
  'sma-100': 0.0,
  'sma-200': 0.0,
  'sma-50': 0.1402165541294383,
  'volume': 0.1033622385225038,
  'fiducia': 

In [26]:
order2 = place_order.place(candle2)
order2

{'ADA': {'order_amt': np.float64(-3851.071775890461),
  'order_price': 0.9979999720448188,
  'stop_price': -0.9062694693014036,
  'stop_portion': np.float64(12104.207019184869),
  'take_price': 3.5370258938397825,
  'take_portion': np.float64(12104.207019184869)},
 'BNB': {'order_amt': np.float64(-1670.281703155426),
  'order_price': 0.9979999999476665,
  'stop_price': 0.6764206411285295,
  'stop_portion': np.float64(12104.206680766183),
  'take_price': 1.4267724783731826,
  'take_portion': np.float64(12104.206680766183)},
 'BTC': {'order_amt': np.float64(-18654.683998582746),
  'order_price': 0.8030588960653758,
  'stop_price': -1.79890861703519,
  'stop_portion': np.float64(41366.82302678383),
  'take_price': 4.272348913532796,
  'take_portion': np.float64(41366.82302678383)},
 'ETH': {'order_amt': np.float64(-16189.482128541225),
  'order_price': 0.947136266077485,
  'stop_price': -0.268071339830637,
  'stop_portion': np.float64(35074.14553050497),
  'take_price': 2.567413073954981,

In [27]:
execute_order.execute(order2)
portfolio = read_file.read_portfolio()
portfolio

,amt,order_price,stop_price,stop_portion,take_price,take_portion
symbol,,,,,,
ETH,50105.922186,0.947136,-0.268071,35074.145531,2.567413,35074.145531
BTC,59095.461467,0.803059,-1.798909,41366.823027,4.272349,41366.823027
BNB,17291.723830,0.998000,0.676421,12104.206681,1.426772,12104.206681
SOL,21134.329131,0.816545,-1.041275,14794.030392,3.293640,14794.030392
ADA,17291.724313,0.998000,-0.906269,12104.207019,3.537026,12104.207019
LINK,17291.723852,0.998000,-0.285853,12104.206697,2.709804,12104.206697
XRP,25398.042112,0.679467,-0.803509,17778.629478,2.656770,17778.629478
LTC,32456.291162,0.531704,-1.568893,22719.403813,3.332500,22719.403813
XLM,24046.306151,0.717663,-0.794998,16832.414306,2.734544,16832.414306


In [28]:
state = read_file.read_state()
state

{'timestep': 3,
 'cash': 119065.83317215217,
 'metrics': {'transaction_cost': 99.5763659150017,
  'returns': 1.0794727484454318,
  'peak equity': 207947.2748445432,
  'drawdown': 0.0}}